In [23]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

import time
import pickle

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
with open('artNouns.dat','rb') as file: # load
    nouns = pickle.load(file)

In [5]:
dic = corpora.Dictionary(nouns)

In [6]:
type(dic)

gensim.corpora.dictionary.Dictionary

In [13]:
corpus = [dic.doc2bow(noun) for noun in nouns] #출현 횟수 vector 구성

In [ ]:
s = time.time()
ldamodel = gensim.models.ldamodel.LdaModel(
                        corpus, num_topics=4, 
                        id2word = dic, 
                        passes=20
                        ) #LDA model 생성
print(time.time()-s)

In [ ]:
ldamodel.save('lda.model')

In [7]:
ldamodel = gensim.models.ldamodel.LdaModel.load('lda.model')

In [8]:
ldamodel.show_topics(num_words=30) #토픽을 구성하는 주요단어들

[(0,
  '0.032*"것" + 0.026*"그" + 0.014*"나" + 0.012*"수" + 0.012*"속" + 0.011*"사람" + 0.011*"우리" + 0.009*"이" + 0.008*"삶" + 0.008*"자신" + 0.008*"모습" + 0.008*"내" + 0.007*"그림" + 0.006*"시간" + 0.005*"이야기" + 0.005*"때" + 0.005*"곳" + 0.005*"세상" + 0.005*"인간" + 0.005*"작가" + 0.005*"마음" + 0.005*"풍경" + 0.004*"기억" + 0.004*"그것" + 0.004*"사진" + 0.004*"일" + 0.004*"말" + 0.004*"작업" + 0.004*"더" + 0.004*"다른"'),
 (1,
  '0.029*"것" + 0.020*"그" + 0.016*"작가" + 0.012*"수" + 0.011*"작업" + 0.011*"작품" + 0.010*"공간" + 0.010*"이미지" + 0.008*"이" + 0.007*"대한" + 0.007*"속" + 0.006*"통해" + 0.006*"인간" + 0.006*"그것" + 0.006*"세계" + 0.005*"자신" + 0.005*"의미" + 0.005*"우리" + 0.005*"다른" + 0.005*"회화" + 0.005*"과정" + 0.005*"현실" + 0.005*"형태" + 0.005*"대상" + 0.004*"하나" + 0.004*"시각" + 0.004*"사물" + 0.004*"시간" + 0.004*"화면" + 0.004*"자연"'),
 (2,
  '0.031*"작가" + 0.025*"전시" + 0.019*"작품" + 0.013*"년" + 0.011*"미술" + 0.011*"이번" + 0.010*"예술" + 0.010*"등" + 0.009*"수" + 0.008*"사진" + 0.008*"통해" + 0.008*"작업" + 0.007*"월" + 0.007*"갤러리" + 0.006*"문화" + 0.006*"이" + 0.006*

#### 참고
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling

In [14]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -7.707110270694123


In [20]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=nouns, dictionary=dic, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3578357027728332


In [24]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dic)
vis

C:\Users\Chankoo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.069437 -0.039298       1        1  43.321880
0     -0.173372 -0.044855       2        1  24.673553
2      0.201557 -0.087188       3        1  17.835234
3      0.041252  0.171340       4        1  14.169333, topic_info=     Category           Freq Term          Total  loglift  logprob
term                                                              
68    Default   33576.000000   전시   33576.000000  30.0000  30.0000
63    Default   69347.000000   작품   69347.000000  29.0000  29.0000
62    Default   92079.000000   작가   92079.000000  28.0000  28.0000
410   Default   24570.000000    나   24570.000000  27.0000  27.0000
4     Default  135092.000000    것  135092.000000  26.0000  26.0000
21    Default   20267.000000    년   20267.000000  25.0000  25.0000
111   Default  106803.000000    그  106803.000000  24.0000  24.0000
33    Default   19108.000000   미술   19108.000000  23.0000  23.0000
257   Default   22667.000000   자연   22667.000000  22.0000  22.0000
405   Default   24053.000000   그림   24053.000000  21.0000  21.0000
230   Default   22666.000000   예술   22666.000000  20.0000  20.0000
57    Default   19490.000000   이번   19490.000000  19.0000  19.0000
180   Default   21099.000000   사람   21099.000000  18.0000  18.0000
312   Default   11826.000000   한국   11826.000000  17.0000  17.0000
141   Default   23625.000000    등   23625.000000  16.0000  16.0000
340   Default   12630.000000    내   12630.000000  15.0000  15.0000
715   Default    9295.000000    꽃    9295.000000  14.0000  14.0000
262   Default    8014.000000   전통    8014.000000  13.0000  13.0000
466   Default   27810.000000  이미지   27810.000000  12.0000  12.0000
183   Default   22058.000000   사진   22058.000000  11.0000  11.0000
796   Default   14762.000000   현대   14762.000000  10.0000  10.0000
3     Default    7556.000000  갤러리    7556.000000   9.0000   9.0000
52    Default    7576.000000    월    7576.000000   8.0000   8.0000
29    Default    9783.000000   문화    9783.000000   7.0000   7.0000
34    Default    6048.000000  미술관    6048.000000   6.0000   6.0000
100   Default   34341.000000   공간   34341.000000   5.0000   5.0000
699   Default   13652.000000   화면   13652.000000   4.0000   4.0000
1469  Default    9141.000000   마음    9141.000000   3.0000   3.0000
1308  Default    4834.000000   아트    4834.000000   2.0000   2.0000
66    Default   10972.000000    전   10972.000000   1.0000   1.0000
...       ...            ...  ...            ...      ...      ...
312    Topic4    6026.758301   한국   11826.460938   1.2800  -4.9399
118    Topic4    3019.200684   기법    5071.257812   1.4355  -5.6312
1978   Topic4    2316.011719    산    3747.830566   1.4728  -5.8963
1003   Topic4    3001.050293   색채    5808.213867   1.2938  -5.6372
63     Topic4   17623.386719   작품   69347.296875   0.5842  -3.8669
405    Topic4    7689.759277   그림   24053.468750   0.8137  -4.6963
916    Topic4    1109.996826   서양    1508.478271   1.6473  -6.6318
1290   Topic4    3306.485596   조형    7792.863281   1.0968  -5.5403
205    Topic4    3052.071533   소재    7553.571289   1.0479  -5.6203
662    Topic4    2570.907227   정신    5874.755859   1.1277  -5.7919
796    Topic4    4536.868164   현대   14762.128906   0.7743  -5.2239
33     Topic4    5276.743164   미술   19108.818359   0.6672  -5.0728
111    Topic4   14722.569336    그  106803.007812  -0.0275  -4.0468
141    Topic4    5798.191895    등   23625.958984   0.5493  -4.9786
189    Topic4    3625.088867    색   10647.552734   0.8766  -5.4483
62     Topic4   11921.350586   작가   92079.054688  -0.0902  -4.2578
21     Topic4    4928.495117    년   20267.949219   0.5401  -5.1411
500    Topic4    3497.242432   표현   11782.040039   0.7395  -5.4842
1183   Topic4    2589.308838    움    6415.245605   1.0468  -5.7848
45     Topic4    7851.698730    수   65118.503906  -0.1614  -4.6754
749    Topic4    4044.676025    선   16926.261719   0.5226  -5.3388
4      Topic4  

![default](https://user-images.githubusercontent.com/38183218/44302152-3e2a1a00-a35e-11e8-90a6-18932f6eb6b7.PNG)

__How to find the optimal number of topics for LDA?__

My approach to finding the optimal number of topics is to build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value.

Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics.

If you see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

The compute_coherence_values() (see below) trains multiple LDA models and provides the models and their corresponding coherence scores.

In [25]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=dic, corpus=corpus, texts=niuns, start=2, limit=5, step=1)

In [ ]:
# Show graph
limit=5; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()